In [39]:
# Import Splinter and BeautifulSoup
from splinter import Browser
from bs4 import BeautifulSoup as soup
import pandas as pd

In [40]:
# Setting executable path - Windows users
executable_path = {'executable_path':r'C:\Users\Jennifer\Desktop\chromedriver_win32\chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

In [42]:
# Visit the mars nasa news site
url = 'https://mars.nasa.gov/news/'
browser.visit(url)
# Optional delay for loading the page
browser.is_element_present_by_css("ul.item_list li.slide", wait_time=1)

True

In [43]:
#Set up HTML parser
html = browser.html
news_soup = soup(html, 'html.parser')
slide_elem = news_soup.select_one('ul.item_list li.slide')

In [44]:
#Assign title and summary text to variables to reference later
slide_elem.find("div", class_='content_title')

<div class="content_title"><a href="/news/8759/nasas-new-mars-rover-will-use-x-rays-to-hunt-fossils/" target="_self">NASA's New Mars Rover Will Use X-Rays to Hunt Fossils</a></div>

In [45]:
# Use the parent element to find the first `a` tag and save it as `news_title`
news_title = slide_elem.find("div", class_='content_title').get_text()
news_title

"NASA's New Mars Rover Will Use X-Rays to Hunt Fossils"

In [46]:
# Use the parent element to find the paragraph text
news_p = slide_elem.find('div', class_="article_teaser_body").get_text()
news_p

"PIXL, an instrument on the end of the Perseverance rover's arm, will search for chemical fingerprints left by ancient microbes."

### Featured Images


In [47]:
# Visit URL
url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(url)

In [48]:
# Find and click the full image button
full_image_elem = browser.find_by_id('full_image')
full_image_elem.click()

In [49]:
# Find the more info button and click that
browser.is_element_present_by_text('more info', wait_time=1)
more_info_elem = browser.links.find_by_partial_text('more info')
more_info_elem.click()

In [50]:
# Parse the resulting html with soup
html = browser.html
img_soup = soup(html, 'html.parser')

In [51]:
# Find the relative image url
img_url_rel = img_soup.select_one('figure.lede a img').get("src")
img_url_rel

'/spaceimages/images/largesize/PIA08097_hires.jpg'

In [52]:
# Use the base URL to create an absolute URL
img_url = f'https://www.jpl.nasa.gov{img_url_rel}'
img_url

'https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA08097_hires.jpg'

In [53]:
#Create dataframe for table_id
df = pd.read_html('http://space-facts.com/mars/')[0]
df.columns=['description', 'value']
df.set_index('description', inplace=True)
df

,value
description,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.39 × 10^23 kg (0.11 Earths)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.38 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-87 to -5 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [35]:
#Dataframe to html
df.to_html()

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>value</th>\n    </tr>\n    <tr>\n      <th>description</th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Equatorial Diameter:</th>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>Polar Diameter:</th>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.39 × 10^23 kg (0.11 Earths)</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <th>Orbit Distance:</th>\n      <td>227,943,824 km (1.38 AU)</td>\n    </tr>\n    <tr>\n      <th>Orbit Period:</th>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <th>Surface Temperature:</th>\n      <td>-87 to -5 °C</td>\n    </tr>\n    <tr>\n      <th>First Record:</th>\n      <td>2nd millennium BC</td>\n    </tr>\n    <tr>\n      <th>Recorded By:</th>\n      <td>Egyptian astronomers</td>\n    </tr>\

### Mars Weather

In [55]:
# Visit the weather website
weather_url = 'https://mars.nasa.gov/insight/weather/'
browser.visit(weather_url)

In [56]:
# Parse the data
html = browser.html
weather_soup = soup(html, 'html.parser')

In [57]:
# Scrape the Daily Weather Report table
weather_table = weather_soup.find('table', class_='mb_table')
print(weather_table.prettify())

<table class="mb_table" id="weather_observation" style="width:100%;">
 <thead>
  <tr>
   <th colspan="2" scope="col">
    Time
   </th>
   <th colspan="3" id="temperature_lbl" scope="col">
    Air Temperature (
    <span class="lbl_fahrenheit">
     °F
    </span>
    <span class="slash">
     |
    </span>
    <span class="lbl_celsius fadeBlack">
     °C
    </span>
    )
   </th>
   <th colspan="4" id="windspeed_lbl" scope="col">
    Wind Speed (
    <span class="lbl_mph">
     mph
    </span>
    <span class="slash">
     |
    </span>
    <span class="lbl_mps fadeBlack">
     m/s
    </span>
    )
   </th>
   <th colspan="3" id="pressure_lbl" scope="col">
    Pressure (Pa)
   </th>
  </tr>
 </thead>
 <tbody>
  <tr id="weather_top">
   <th class="sol" scope="row">
    Date
   </th>
   <th class="sol" scope="row">
    Sol
   </th>
   <td class="temperature max">
    Max.
   </td>
   <td class="temperature avg">
    Avg.
   </td>
   <td class="temperature min">
    Min.
   </td>
   <t

## D1: Scrape High-Resolution Mars' Hemishphere Images and Titles

In [78]:
# 1. Use browser to visit the URL 
hemi_url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(hemi_url)

In [84]:
# 2. Create a list to hold the images and titles.
hemisphere_image_urls = []

In [64]:
# 3. Write code to retrieve the image urls and titles for each hemisphere.


[]

In [85]:
# 4. Print the list that holds the dictionary of each image url and title.
hemisphere_image_urls

[]

In [ ]:
# 5. Quit the browser
browser.quit()